# PERSIAPKAN DATA

Membersihkan Lingkungan Dari Data Sebelumnya

In [1]:
import os, shutil

if os.path.exists('data.zip'): os.remove('data.zip')
if os.path.exists('data'): shutil.rmtree('data')

Download Data Yang Akan Dijadikan Dataset

In [2]:
import os, gdown
from zipfile import ZipFile

gdown.download('https://drive.google.com/uc?id=1w4p9ugXJrUV-qmxt6UyJZLAzj0uefIvZ', 'data.zip', quiet=False)

if not os.path.exists('data'): os.makedirs("data")

with ZipFile('data.zip', 'r') as zObject:
	zObject.extractall('data')

Downloading...
From (original): https://drive.google.com/uc?id=1w4p9ugXJrUV-qmxt6UyJZLAzj0uefIvZ
From (redirected): https://drive.google.com/uc?id=1w4p9ugXJrUV-qmxt6UyJZLAzj0uefIvZ&confirm=t&uuid=961adc40-7766-4f86-9f61-5685b8a23c2c
To: d:\Model Pembelajaran Mesin\klasifikasi-gambar-buaya-dan-kura-kura\data.zip
100%|██████████| 62.9M/62.9M [01:03<00:00, 993kB/s] 


# AUGMENTASI GAMBAR

Generator untuk membuat gambar menjadi lebih variatif dan menentukan besar data yang digunakan untuk validasi

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    horizontal_flip = True,
    shear_range = 0.2,
    fill_mode = 'nearest',
    validation_split = 0.2
)

# DATASET

Ubah Data Menjadi Dataset Training & Validation

In [4]:
lokasi = 'data'

training = datagen.flow_from_directory(
    lokasi,
    target_size = (50, 50),
    class_mode = 'binary',
    subset = 'training'
)

validation = datagen.flow_from_directory(
    lokasi,
    target_size = (50, 50),
    class_mode = 'binary',
    subset = 'validation'
)

Found 2044 images belonging to 2 classes.
Found 510 images belonging to 2 classes.


# MODEL CNN

Build Model Sequential Untuk Klasifikasi 2 Gambar (biner) dengan 3 layer neuron dan 1 output

In [5]:
from tensorflow import keras

model = keras.Sequential()

model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 3)))
model.add(keras.layers.MaxPool2D(2, 2))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPool2D(2, 2))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(keras.layers.MaxPool2D(2, 2))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Flatten())
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 24, 24, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 24, 24, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 11, 11, 64)        0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 11, 11, 64)       

# LATIH

Model Yang Sudah Dibuat Dilatih Memakai Dataset Training Dan Validation

In [6]:
import tensorflow as tf

accuracy_threshold = 85e-2
class my_callbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = None):
        if logs.get('val_accuracy') >= accuracy_threshold:
            self.model.stop_training = True
            
history = model.fit(training, steps_per_epoch=64, epochs=100, validation_data=validation, callbacks=[my_callbacks()])

Epoch 1/100


64/64 [==============================] - 32s 445ms/step - loss: 0.7265 - accuracy: 0.6600 - val_loss: 0.6560 - val_accuracy: 0.6765
Epoch 2/100
64/64 [==============================] - 10s 157ms/step - loss: 0.6607 - accuracy: 0.6766 - val_loss: 0.5878 - val_accuracy: 0.7294
Epoch 3/100
64/64 [==============================] - 10s 157ms/step - loss: 0.5655 - accuracy: 0.7197 - val_loss: 0.5550 - val_accuracy: 0.7314
Epoch 4/100
64/64 [==============================] - 11s 172ms/step - loss: 0.5162 - accuracy: 0.7461 - val_loss: 0.5456 - val_accuracy: 0.7314
Epoch 5/100
64/64 [==============================] - 12s 187ms/step - loss: 0.4966 - accuracy: 0.7593 - val_loss: 0.5036 - val_accuracy: 0.7392
Epoch 6/100
64/64 [==============================] - 10s 161ms/step - loss: 0.4830 - accuracy: 0.7657 - val_loss: 0.6691 - val_accuracy: 0.5627
Epoch 7/100
64/64 [==============================] - 11s 163ms/step - loss: 0.4717 - accuracy: 0.7828 - val_loss: 0.4408 - val_accurac

Grafik Riwayat Pelatihan

In [ ]:
import matplotlib.pyplot as plt

accuracy = history.history['val_accuracy']
loss = history.history['val_loss']

plt.plot(accuracy, color='blue', label='Akurasi')
plt.plot(loss, color='red', label='Loss')
plt.xlabel('Epoch')
plt.ylabel('Nilai')
plt.title('Riwayat Pelatihan Model')
plt.legend()
plt.show()

Cek Akurasi Model

In [ ]:
eval_result = model.evaluate(validation)

accuracy = eval_result[1]
print(f'\nAkurasi Model: {accuracy * 100:.2f} %')

# Evaluasi

Menguji Model Dengan Berbagai Kemungkinan Gambar Di Luar Dataset

In [ ]:
import os, numpy as np, matplotlib.pyplot as plt
from keras.preprocessing import image

daftar_file = os.listdir('evaluasi')
daftar_file_urut = sorted(daftar_file, key=lambda x: int(os.path.splitext(x)[0]))

if os.path.exists('evaluasi'):
    for file in daftar_file_urut:
        nama_file = 'evaluasi/' + file 

        display = image.load_img(nama_file, target_size=None)
        plt.imshow(display)

        img = image.load_img(nama_file, target_size=(50, 50))

        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)

        predictions = model.predict(img_array)

        if predictions[0][0] > 0.5:
            plt.xlabel(f'KURA - KURA', fontsize=15)
        else:
            plt.xlabel(f'BUAYA', fontsize=15)
        plt.show()
else :
    print('Silahkan buat folder evaluasi dan isi dengan gambar buaya dan kura kura acak terlebih dulu')
    exit